# Pythonic Vector2D

In [73]:
from array import array
import math

class Vector2d:
    typecode = 'd' # 类属性，在Vector2d实例和字节序列之间转换时使用
    
    def __init__(self, x, y):
        self.x = float(x) # 提前检查数据类型
        self.y = float(y)
        
    def __iter__(self): # 转换成可迭代对象方便拆包（x, y = my_vector）
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self): # 模是x和y分量构成的直角三角形的斜边长 
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

In [74]:
v1 = Vector2d(3, 4)

In [75]:
print(v1)

(3.0, 4.0)


In [76]:
v1

Vector2d(3.0, 4.0)

In [77]:
repr(v1)

'Vector2d(3.0, 4.0)'

In [78]:
v1_clone = eval(repr(v1)) # eval(): 执行一个字符串表达式并返回结果

In [79]:
v1 == v1_clone

True

In [80]:
octets = bytes(v1)

In [81]:
abs(v1)

5.0

In [82]:
bool(Vector2d(0, 0))

False

## 备选构造方法
添加一个功能，从字节序列转换为Vector2d实例

重写array.array的类方法：`.frombytes`

In [83]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self): 
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [84]:
v2 = Vector2d(5, 6)

In [85]:
v2.frombytes(octets)

Vector2d(3.0, 4.0)

In [86]:
Vector2d.frombytes(octets)

Vector2d(3.0, 4.0)

## classmethod和staticmethod

`classmethod`操作类本身，最常见的用处是定义备选构造方法（如上），而`staticmethod`的行为和普通函数类似，通常没啥用。

反驳观点推荐阅读：The definitive guide on how to use static, class or abstract methods in Python (https://julien.danjou.info/guide-python-static-class-abstract-methods/ )

事实上由于`staticmethod`不需要绑定给任何实例，在一些使用过程中可以避免类方法绑定给实例的开销：unbounded function to bounded function

# Hashable Vector2d

为了将实例放入集合或者左右dict的key，实例需要是可散列的。为此必须实现`__hash__`和`__eq__`方法，还要让向量不可变。

使用两个前导下划线将属性标记为私有的，并使用`@property`将读值方法标记为特性。（读值方法与公开属性同名）

In [87]:
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = x
        self.__y = y
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __eq__(self, other):
        return tuple(self) == tuple(other)

In [88]:
a = Vector2d(3, 4)

In [89]:
a.__x

AttributeError: 'Vector2d' object has no attribute '__x'

In [90]:
a.x

3

可以使用`_class-name__attr`来对私有属性进行访问

In [91]:
a._Vector2d__x

3

In [92]:
a._Vector2d__x = 6

In [93]:
a.x

6

事实上，使用两个前导下划线将属性标记为私有并不能阻止对此变量的访问（知道改写其的机制的话），通常情况下我们约定使用一个前导下划线来标记不希望从外部访问的属性。

完整的Vector2d示例如下（未实现`__format__`）方法：

In [96]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = x
        self.__y = y
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, self)))

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self): 
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

# `__slots__`

使用`__slots__`类属性可以节省空间：

- 实例只能包含在`__slots__`里面定义的属性
- 解释器会在各个实例中使用相似的结构来存储实例变量，这样能节省大量内存空间

In [97]:
class Vector2d:
    __slots__ = ('__x', '__y')
    
    typecode = 'd'
    
    '''方法实现略'''

如果把`__dict__`添加到`__slots__`中，实例就能支持动态创建属性——这样违背了优化的初衷。

使用`__slots__`要注意：
- 每个子类都要定义`__slots__`属性，因为解释器会忽略继承`__slots__`的属性
- 实例只能拥有`__slots__`中的属性
- 如果不把`__weakref__`加入`__slots__`，实例就不能作为弱引用的目标